This notebook completely reproduces `Aeff` production from CTA-KM3NeT combined analysis  using somefuctioanlity from  `km3irf` 

In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import km3io

from astropy.io import fits
import astropy.units as u

from gammapy.irf import EnergyDispersion2D

from scipy.stats import binned_statistic
from scipy.ndimage import gaussian_filter1d, gaussian_filter

# from scipy.ndimage.filters import gaussian_filter1d, gaussian_filter

import pandas as pd
import uproot
from collections import defaultdict

import sys

sys.path.append("./ppflux")
sys.path.append("../")
from km3irf.irf_tools import calc_theta, aeff_2D, psf_3D, edisp_3D

# import flux as km3_flux
# import plot_utils

In [2]:
# normal data with bdt
filename_nu = "../../data_files/files_cta_km3net/mcv5.1.km3_numuCC.ALL.dst.bdt.root"
filename_nubar = "../../data_files/files_cta_km3net/mcv5.1.km3_anumuCC.ALL.dst.bdt.root"
filename_mu10 = (
    "../../data_files/files_cta_km3net/mcv5.2.mupage_10T.sirene_mupage.ALL.bdt.root"
)
filename_mu50 = (
    "../../data_files/files_cta_km3net/mcv5.2.mupage_50T.sirene_mupage.ALL.bdt.root"
)

In [3]:
# reading with km3io
f_nu_km3io = km3io.OfflineReader(filename_nu)
f_nubar_km3io = km3io.OfflineReader(filename_nubar)

In [4]:
# Print header info
print("nu:")
print(f_nu_km3io.header.ngen)
print(f_nu_km3io.header.genvol)
print(f_nu_km3io.header.spectrum)
print("\nnubar:")
print(f_nubar_km3io.header.ngen)
print(f_nubar_km3io.header.genvol)
print(f_nubar_km3io.header.spectrum)

nu:
100000.0
genvol(zmin=0, zmax=1027, r=888.4, volume=2649000000.0, numberOfEvents=20000000.0)
spectrum(alpha=-1.4)

nubar:
100000.0
genvol(zmin=0, zmax=1027, r=888.4, volume=2649000000.0, numberOfEvents=20000000.0)
spectrum(alpha=-1.4)


In [5]:
%%time
# Access data arrays
data_km3io = dict()

for l, f in zip(["nu", "nubar"], [f_nu_km3io, f_nubar_km3io]):
    data_km3io[l] = dict()

    data_km3io[l]["E"] = f.tracks.E.to_numpy()[:, 0]
    data_km3io[l]["dir_x"] = f.tracks.dir_x.to_numpy()[:, 0]
    data_km3io[l]["dir_y"] = f.tracks.dir_y.to_numpy()[:, 0]
    data_km3io[l]["dir_z"] = f.tracks.dir_z.to_numpy()[:, 0]

    data_km3io[l]["E_mc"] = f.mc_tracks.E.to_numpy()[:, 0]
    data_km3io[l]["dir_x_mc"] = f.mc_tracks.dir_x.to_numpy()[:, 0]
    data_km3io[l]["dir_y_mc"] = f.mc_tracks.dir_y.to_numpy()[:, 0]
    data_km3io[l]["dir_z_mc"] = f.mc_tracks.dir_z.to_numpy()[:, 0]

    data_km3io[l]["weight_w2"] = f.w.to_numpy()[:, 1]

Wall time: 1.82 s


Read files with uproot

In [6]:
f_nu_uproot = uproot.open(filename_nu)
f_nubar_uproot = uproot.open(filename_nubar)

In [7]:
%%time
# Retrieve data
data_uproot = dict()

for l, f in zip(["nu", "nubar"], [f_nu_uproot, f_nubar_uproot]):
    # Retrieve trees
    E = f["E;1"]
    T = f["T;1"]

    # Access data arrays
    data_uproot[l] = dict()

    data_uproot[l]["E"] = E["Evt/trks/trks.E"].array().to_numpy()[:, 0]
    data_uproot[l]["dir_x"] = E["Evt/trks/trks.dir.x"].array().to_numpy()[:, 0]
    data_uproot[l]["dir_y"] = E["Evt/trks/trks.dir.y"].array().to_numpy()[:, 0]
    data_uproot[l]["dir_z"] = E["Evt/trks/trks.dir.z"].array().to_numpy()[:, 0]

    data_uproot[l]["E_mc"] = E["Evt/mc_trks/mc_trks.E"].array().to_numpy()[:, 0]
    data_uproot[l]["dir_x_mc"] = E["Evt/mc_trks/mc_trks.dir.x"].array().to_numpy()[:, 0]
    data_uproot[l]["dir_y_mc"] = E["Evt/mc_trks/mc_trks.dir.y"].array().to_numpy()[:, 0]
    data_uproot[l]["dir_z_mc"] = E["Evt/mc_trks/mc_trks.dir.z"].array().to_numpy()[:, 0]

    data_uproot[l]["weight_w2"] = E["Evt/w"].array().to_numpy()[:, 1]
    bdt = T["bdt"].array().to_numpy()
    data_uproot[l]["bdt0"] = bdt[:, 0]
    data_uproot[l]["bdt1"] = bdt[:, 1]

Wall time: 2.19 s


In [8]:
# Make sure we have read the same arrays with km3io and uproot
for l in ["nu", "nubar"]:
    print(l)
    for k in data_km3io[l].keys():
        print(k, np.all(data_km3io[l][k] == data_uproot[l][k]))
    print("")

nu
E True
dir_x True
dir_y True
dir_z True
E_mc True
dir_x_mc True
dir_y_mc True
dir_z_mc True
weight_w2 True

nubar
E True
dir_x True
dir_y True
dir_z True
E_mc True
dir_x_mc True
dir_y_mc True
dir_z_mc True
weight_w2 True



In [9]:
# Create pandas DataFrames
df_nu = pd.DataFrame(data_uproot["nu"])
df_nubar = pd.DataFrame(data_uproot["nubar"])

Selection cuts

In [11]:
# Implement selection cuts
def get_q_mask(bdt0, bdt1, dir_z):
    """
    bdt0: to determine groups to which BDT cut should be applied (upgoing/horizontal/downgoing).
    bdt1: BDT score in the range [-1, 1]. Closer to 1 means more signal-like.
    dir_z: is the reconstructed z-direction of the event
    """
    mask_down = bdt0 >= 11  # remove downgoing events
    clear_signal = bdt0 == 12  # very clear signal
    loose_up = (np.arccos(dir_z) * 180 / np.pi < 80) & (
        bdt1 > 0.0
    )  # apply loose cut on upgoing events
    strong_horizontal = (np.arccos(dir_z) * 180 / np.pi > 80) & (
        bdt1 > 0.7
    )  # apply strong cut on horizontal events
    return mask_down & (clear_signal | loose_up | strong_horizontal)

In [12]:
# Apply the cuts

# nu
q_mask_nu = get_q_mask(df_nu.bdt0, df_nu.bdt1, df_nu.dir_z)
df_nu_q = df_nu[q_mask_nu].copy()

# nubar
q_mask_nubar = get_q_mask(df_nubar.bdt0, df_nubar.bdt1, df_nubar.dir_z)
df_nubar_q = df_nubar[q_mask_nubar].copy()

In [13]:
print(
    "nu: {:d} events survive cuts ({:.4g}%)".format(
        len(df_nu_q), len(df_nu_q) / len(df_nu) * 100
    )
)
print(
    "nubar: {:d} events survive cuts ({:.4g}%)".format(
        len(df_nubar_q), len(df_nubar_q) / len(df_nubar) * 100
    )
)

nu: 331325 events survive cuts (41.27%)
nubar: 362186 events survive cuts (42.86%)


In [14]:
# calculate the normalized weight factor for each event
weight_factor = -2.5  # Spectral index to re-weight to

weights = dict()
for l, df, f in zip(
    ["nu", "nubar"], [df_nu_q, df_nubar_q], [f_nu_km3io, f_nubar_km3io]
):
    weights[l] = (df.E_mc ** (weight_factor - f.header.spectrum.alpha)).to_numpy()
    weights[l] *= len(df) / weights[l].sum()

In [15]:
# Create DataFrames with neutrinos and anti-neutrinos
df_nu_all = pd.concat([df_nu, df_nubar], ignore_index=True)
df_nu_all_q = pd.concat([df_nu_q, df_nubar_q], ignore_index=True)

# Also create a concatenated array for the weights
weights_all = np.concatenate([weights["nu"], weights["nubar"]])

In [16]:
# Define bins for IRFs
cos_bins_fine = np.linspace(1, -1, 13)
t_bins_fine = np.arccos(cos_bins_fine)
e_bins_fine = np.logspace(2, 8, 49)

cos_bins_coarse = np.linspace(1, -1, 7)
t_bins_coarse = np.arccos(cos_bins_coarse)
e_bins_coarse = np.logspace(2, 8, 25)

migra_bins = np.logspace(-5, 2, 57)
rad_bins = np.concatenate(
    (
        np.linspace(0, 1, 21),
        np.linspace(1, 5, 41)[1:],
        np.linspace(5, 30, 51)[1:],
        [180.0],
    )
)

In [17]:
# Bin centers
e_binc_fine = np.sqrt(e_bins_fine[:-1] * e_bins_fine[1:])
e_binc_coarse = np.sqrt(e_bins_coarse[:-1] * e_bins_coarse[1:])

t_binc_fine = np.arccos(0.5 * (cos_bins_fine[:-1] + cos_bins_fine[1:]))
t_binc_coarse = np.arccos(0.5 * (cos_bins_coarse[:-1] + cos_bins_coarse[1:]))

migra_binc = np.sqrt(migra_bins[:-1] * migra_bins[1:])
rad_binc = 0.5 * (rad_bins[1:] + rad_bins[:-1])

In [18]:
# fill aeff histogram
aeff = (
    aeff_2D(
        e_bins=e_bins_fine,
        t_bins=t_bins_fine,
        dataset=df_nu_all_q,
        gamma=(-weight_factor),
        nevents=f_nu_km3io.header.genvol.numberOfEvents
        + f_nubar_km3io.header.genvol.numberOfEvents
        # nevents = df_nu_all_q.shape[0]
    )
    * 2
)  # two building blocks

In [19]:
f_nu_km3io.header.genvol.numberOfEvents + f_nubar_km3io.header.genvol.numberOfEvents

40000000.0

In [21]:
df_nu_all_q.shape[0]

693511

In [22]:
f_nu_km3io.header.genvol.numberOfEvents

20000000.0

write Aeff to `.fits`

In [19]:
hdu = fits.PrimaryHDU()

In [20]:
# Write AEFF
col1 = fits.Column(
    name="ENERG_LO",
    format="{}E".format(len(e_binc_fine)),
    unit="GeV",
    array=[e_bins_fine[:-1]],
)
col2 = fits.Column(
    name="ENERG_HI",
    format="{}E".format(len(e_binc_fine)),
    unit="GeV",
    array=[e_bins_fine[1:]],
)
col3 = fits.Column(
    name="THETA_LO",
    format="{}E".format(len(t_binc_fine)),
    unit="rad",
    array=[t_bins_fine[:-1]],
)
col4 = fits.Column(
    name="THETA_HI",
    format="{}E".format(len(t_binc_fine)),
    unit="rad",
    array=[t_bins_fine[1:]],
)
col5 = fits.Column(
    name="EFFAREA",
    format="{}D".format(len(e_binc_fine) * len(t_binc_fine)),
    dim="({},{})".format(len(e_binc_fine), len(t_binc_fine)),
    unit="m2",
    array=[aeff.T],
)
cols = fits.ColDefs([col1, col2, col3, col4, col5])
hdu2 = fits.BinTableHDU.from_columns(cols)
hdu2.header["EXTNAME"] = "EFFECTIVE AREA"
hdu2.header[
    "HDUDOC"
] = "https://github.com/open-gamma-ray-astro/gamma-astro-data-formats"
hdu2.header["HDUVERS"] = "0.2"
hdu2.header["HDUCLASS"] = "GADF"
hdu2.header["HDUCLAS1"] = "RESPONSE"
hdu2.header["HDUCLAS2"] = "EFF_AREA"
hdu2.header["HDUCLAS3"] = "FULL-ENCLOSURE"
hdu2.header["HDUCLAS4"] = "AEFF_2D"
aeff_fits = fits.HDUList([hdu, hdu2])
aeff_fits.writeto("aeff_nevents.fits", overwrite=True)

In [21]:
type(f_nu_km3io)

km3io.offline.OfflineReader

In [22]:
a = f_nu_km3io + f_nubar_km3io

TypeError: unsupported operand type(s) for +: 'OfflineReader' and 'OfflineReader'